### Predição do conjunto de dados m_694.csv com o modelo LSTM

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.random import set_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, max_error, mean_absolute_error

In [2]:
# Carregar dados do CPU
machine = 'm_694.csv'
workload = pd.read_csv(machine, 
             names = ['machine_id', 'time_stamp', 'cpu', 'mem', 'mem_gps', 
                      'mkpi', 'net_in', 'net_out', 'disk_io_percent'])
workload.time_stamp = pd.to_datetime(workload.time_stamp, unit='s')
workload.set_index('time_stamp', inplace=True)
workload = workload.resample('5min').interpolate()
workload = workload[-12*24*3:] # 5min * 12 * 24 * 3 = 3 days
workload

,machine_id,cpu,mem,mem_gps,mkpi,net_in,net_out,disk_io_percent
time_stamp,,,,,,,,
1970-01-06 00:00:00,m_694,25.0,96.0,NaN,NaN,46.14,39.77,5.0
1970-01-06 00:05:00,m_694,31.0,97.0,NaN,NaN,46.15,39.78,5.0
1970-01-06 00:10:00,m_694,34.0,94.0,NaN,NaN,46.15,39.78,8.0
1970-01-06 00:15:00,m_694,31.0,92.0,NaN,NaN,46.16,39.79,4.0
1970-01-06 00:20:00,m_694,32.0,94.0,NaN,NaN,46.16,39.79,4.0
...,...,...,...,...,...,...,...,...
1970-01-08 23:35:00,m_694,86.0,91.0,0.35,0.0,51.46,44.74,4.0
1970-01-08 23:40:00,m_694,86.0,92.0,0.27,0.0,51.46,44.75,6.0
1970-01-08 23:45:00,m_694,85.0,92.0,0.45,0.0,51.46,44.75,6.0


In [3]:
# Grid search LSTM
# Source: https://machinelearningmastery.com/how-to-grid-search-deep-learning-models-for-time-series-forecasting/

import datetime
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from tensorflow.random import set_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# transform list into supervised learning format
def series_to_supervised(data_arr, look_back):
    X, Y =[], []
    for i in range(len(data_arr)-look_back):
        d=i+look_back
        X.append(data_arr[i:d,])
        Y.append(data_arr[d,])
    return np.array(X), np.array(Y)

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# difference dataset
def difference(data, order):
    return [data[i] - data[i - order] for i in range(order, len(data))]

# fit a model
def model_fit(train, config):
    # unpack config
    n_input, n_nodes, n_epochs, n_batch, n_diff = config
    # prepare data
    if n_diff > 0:
        train = difference(train, n_diff)
    # transform series into supervised format
    #data = series_to_supervised(train, n_in=n_input)
    # separate inputs and outputs
    train_x, train_y = series_to_supervised(train, n_input)#data[:, :-1], data[:, -1]
    # reshape input data into [samples, timesteps, features]
    n_features = 1
    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
    # define model
    model = Sequential()
    model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    # fit model
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
    return model

# forecast with the fit model
def model_predict(model, history, config):
    # unpack config
    n_input, _, _, _, n_diff = config
    # prepare data
    correction = 0.0
    if n_diff > 0:
        correction = history[-n_diff]
        history = difference(history, n_diff)
    # reshape sample into [samples, timesteps, features]
    x_input = array(history[-n_input:]).reshape((1, n_input, 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    
    # reverse scale
    predictions = scaler.inverse_transform(predictions)
    test = scaler.inverse_transform(test)
    
    # estimate prediction error
    error = measure_rmse(test, predictions)
    #print(' > %.3f' % error)
    return error

# score a model, return None on failure
def repeat_evaluate(data, config, n_test):
    # convert config to a key
    key = str(config)
    # fit and evaluate the model n times
    scores = [walk_forward_validation(data, n_test, config) for _ in range(config[2])]
    # summarize score
    result = mean(scores)
    print('> Model[%s] %.3f' % (key, result))
    return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
    # evaluate configs
    scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

# create a list of configs to try
def model_configs():
    # define scope of configs
    n_input = [12]
    n_nodes = [14,15,16]
    n_epochs = [150,200,300]
    n_batch = [20,25,30,40]
    n_diff = [0]
    # create configs
    configs = list()
    for i in n_input:
        for j in n_nodes:
            for k in n_epochs:
                for l in n_batch:
                    for m in n_diff:
                        cfg = [i, j, k, l, m]
                        configs.append(cfg)
    print('Total configs: %d' % len(configs))
    return configs

# define dataset
data = workload[['cpu']].values
scaler = MinMaxScaler(feature_range=(0, 1))#LTSM is senstive to the scale of features
data = scaler.fit_transform(data)

# disable random weigths
np.random.seed(1234)
set_seed(1234)

# data split
n_test = int(len(workload)*1/3)
# model configs
cfg_list = model_configs()
# grid search
start = datetime.datetime.now()
scores = grid_search(data, cfg_list, n_test)
print('done')
lapsed = datetime.datetime.now() - start
print('Tempo: '+str(lapsed).split('.')[0])
# list top 10 configs
for cfg, error in scores[:10]:
    print(cfg, error)

Total configs: 36
> Model[[12, 14, 150, 20, 0]] 7.375
> Model[[12, 14, 150, 25, 0]] 7.324
> Model[[12, 14, 150, 30, 0]] 7.203
> Model[[12, 14, 150, 40, 0]] 7.257
> Model[[12, 14, 200, 20, 0]] 7.676
> Model[[12, 14, 200, 25, 0]] 7.790
> Model[[12, 14, 200, 30, 0]] 7.643
> Model[[12, 14, 200, 40, 0]] 7.466
> Model[[12, 14, 300, 20, 0]] 8.039
> Model[[12, 14, 300, 25, 0]] 7.859
> Model[[12, 14, 300, 30, 0]] 7.871
> Model[[12, 14, 300, 40, 0]] 7.638
> Model[[12, 15, 150, 20, 0]] 7.398
> Model[[12, 15, 150, 25, 0]] 7.369
> Model[[12, 15, 150, 30, 0]] 7.275
> Model[[12, 15, 150, 40, 0]] 7.295
> Model[[12, 15, 200, 20, 0]] 7.754
> Model[[12, 15, 200, 25, 0]] 7.705
> Model[[12, 15, 200, 30, 0]] 7.635
> Model[[12, 15, 200, 40, 0]] 7.428
> Model[[12, 15, 300, 20, 0]] 7.896
> Model[[12, 15, 300, 25, 0]] 7.921
> Model[[12, 15, 300, 30, 0]] 7.835
> Model[[12, 15, 300, 40, 0]] 7.585
> Model[[12, 16, 150, 20, 0]] 7.486
> Model[[12, 16, 150, 25, 0]] 7.281
> Model[[12, 16, 150, 30, 0]] 7.173
> Model[[1